here i attempt to write a REST2 factory for a canonical openmm.system (i.e. containing `HarmonicBondForce`, `HarmonicAngleForce`, `PeriodicTorsionForce`, `NonbondedForce`)

In [1]:
from openmmtools.testsystems import AlanineDipeptideVacuum

define a vacuum alanine dipeptide

In [2]:
ala = AlanineDipeptideVacuum()
sys = ala.system

In [3]:
from perses.annihilation.rest import RESTTopologyFactory

query the topology

In [4]:
ala.topology
for atom in ala.topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>


I'll make the ALA residue 'solute' for REST2

remove the `CMMForce`

In [5]:
ala.system.removeForce(4)

make the factory

In [6]:
factory = RESTTopologyFactory(ala.system, solute_region=list(range(6, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


pull the system

In [7]:
REST_system = factory.REST_system

query the forces to make sure it _looks_ right

In [8]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2afe1d9d3f00> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2afe1d9d3f90> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2afe1d9d3ed0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2afe1d9d3e70> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2afe1d9d3ea0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2afdec85a8d0> >]

all of the nonbonded forces must have the same number of particles

In [9]:
num_particles = REST_system.getNumParticles()
assert num_particles == factory._out_system_forces['CustomNonbondedForce'].getNumParticles()
assert num_particles == factory._out_system_forces['NonbondedForce'].getNumParticles()

## TODO:

here is a few things that have to be done...

- Given the default Global variables, do energy component bookkeeping with the REST system and the original system. Specifically, the first `CustomBondForce` must match the `HarmonicBondForce` (same with the second and third forces with the `HarmonicAngle` and `PeriodicTorsionForce`s). The sum of the `NonbondedForce`, the `CustomNonbondedForce`, and the `CustomBondForce` (last one) must be equal to the energy of the original system's `NonbondedForce`.

- Create a subclass of the `AlchemicalState` to implement the appropriate parameters for `solute_scale` and `inter_scale` (which are `GlobalParameters` of the `REST_system`) and perform Repex.

- Run REST2!

## Run energy component bookkeeping

In [10]:
from openmmtools.states import CompoundThermodynamicState, SamplerState, ThermodynamicState
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from openmmtools.integrators import LangevinIntegrator
from simtk import unit
from openmmtools import cache
temp = 300 * unit.kelvin
from perses.tests.utils import compute_potential_components
import copy
from openmmtools.constants import kB
beta = 1/(kB * temp)
from perses.dispersed.feptasks import minimize
from coddiwomple.openmm.states import OpenMMPDFState
from perses.dispersed.utils import configure_platform
from coddiwomple.openmm.utils import get_dummy_integrator
from openmmtools import utils

conducting subsequent work with the following platform: CUDA


In [11]:
from perses.tests.test_topology_proposal import generate_atp

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


Potential components for the vanilla ala system

In [12]:
thermostate = ThermodynamicState(system=ala.system, temperature=temp)
sampler_state = SamplerState(ala.positions, box_vectors = ala.system.getDefaultPeriodicBoxVectors())
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
minimize(thermostate, sampler_state)
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.5874003195280203), ('HarmonicAngleForce', 1.209485556878217), ('PeriodicTorsionForce', 2.321496276111238), ('NonbondedForce', -50.94914523261006), ('AndersenThermostat', 0.0)]


Potential components for the REST system

In [13]:
thermostate = ThermodynamicState(system=REST_system, temperature=temp)
# Don't re-initialize the sampler state 
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
# Don't minimize again
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.5874003195280203), ('CustomAngleForce', 1.2094855568782168), ('CustomTorsionForce', 2.3214942161284062), ('NonbondedForce', -1.187543685095446), ('CustomNonbondedForce', -133.23778625838835), ('CustomBondForce', 83.4761858046491), ('AndersenThermostat', 0.0)]


In [ ]:
# REST Nonbonded + CustomNonbonded + CustomBond = -50.949